<a href="https://colab.research.google.com/github/mcash8/DVBS2X-DNN-Implementation/blob/main/Feed%20Forward%20NN/Doppler%20Effect/NN_CFO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Mount Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Imports

In [2]:
import os
import scipy.io as spio
import math
import time
import pandas as pd
from pandas import read_csv
import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.losses import MeanSquaredError

In [ ]:
#Train model
%cd /content/drive/My Drive/NN\ Input/Doppler/Train\ Data
start_time = time.time()

#parameters
num_classes = 2
batch_size = 64
epochs = 100

ratios = list(range(1,16))

for ratio in ratios: 
    matname = 'DVBS_dopp_ratio_%d' % (ratio)
    print(matname)
    mat = spio.loadmat(matname, squeeze_me=True)
    x_train = mat['x_train']
    x_valid = mat['val_data']
    y_valid = mat['val_target']
    y_train = mat['y_train']


    #cast to float32
    x_train = x_train.astype('float32')
    x_valid = x_valid.astype('float32')
    #x_test = x_test.astype('float32') 

    print(x_train.shape, 'train samples')
    print(x_valid.shape, 'valid samples')

    print(y_train.shape, 'train labels')
    print(y_valid.shape, 'valid labels')

    print('Label Examples:\n', y_train)
    print('Input Examples:\n', x_train)

    # Define the network
    model = Sequential()
    model.add(Dense(50, activation='elu', input_dim=18))
    model.add(Dense(50, activation='elu'))
    model.add(Dense(num_classes, activation='tanh'))

    model.summary()

    model.compile(loss=keras.metrics.mean_squared_error,
                optimizer=SGD(learning_rate=0.1),
                metrics=[keras.metrics.RootMeanSquaredError(name='rmse')])

    history = model.fit(x_train, y_train,
                        batch_size=batch_size,
                        epochs=epochs,
                        verbose=2,
                        validation_data=(x_valid, y_valid))

    elapsedTime = (time.time() - start_time)/60
    print("--- %.2f minutes ---" % elapsedTime)

    savename = 'model_dopp_ratio_%d.h5' % (ratio)
    model.save(savename)

Test trained models. Test sequence is same length and rotation as train data.

In [ ]:
ratios = list(range(2,16))

for ratio in ratios:
    model_name = 'model_dopp_ratio_%d.h5' % (ratio)
    folder_name = 'ratio_%d' % (ratio)

    #load trained model 
    %cd /content/drive/My Drive/NN\ Input/Doppler/Train\ Data
    model = tf.keras.models.load_model(model_name)

    %cd /content/drive/My Drive/NN\ Input/Doppler/Test\ Data
    
    %cd $folder_name 
    
    for i in range(1,11): #loop over 10 test sequences 
        matname = 'DVBSAPSK_pred_ratio_%d_%d' % (ratio, i)

        mat = spio.loadmat(matname, squeeze_me=True) #load data
        x_test = mat['train_data']

        predictions = model.predict(x_test) #make predictions

        #save predictions 
        matname = 'DVBSAPSK_out_ratio_%d_%d.mat' % (ratio, i)
        spio.savemat(matname, {'pred': predictions})

    %cd ..
    

Test if sequence length significantly matters in NN output

In [ ]:
ratios = list(range(1,16))
#two foor loops, start with short length first, then long length 

main_folder = 'short_len'
for ratio in ratios:
    model_name = 'model_dopp_ratio_%d.h5' % (ratio)
    folder_name = 'ratio_%d' % (ratio)

    #load trained model 
    %cd /content/drive/My Drive/NN\ Input/Doppler/Train\ Data
    model = tf.keras.models.load_model(model_name)

    %cd /content/drive/My Drive/NN\ Input/Doppler/Test\ Data

    %cd $main_folder
    %cd $folder_name 
    
    for i in range(1,11): #loop over 10 test sequences 
        matname = 'DVBSAPSK_pred_ratio_%d_%d' % (ratio, i)

        mat = spio.loadmat(matname, squeeze_me=True) #load data
        x_test = mat['train_data']

        predictions = model.predict(x_test) #make predictions

        #save predictions 
        matname = 'DVBSAPSK_out_ratio_%d_%d.mat' % (ratio, i)
        spio.savemat(matname, {'pred': predictions})

    %cd ..
    %cd ..


main_folder = 'long_len'
for ratio in ratios:
    model_name = 'model_dopp_ratio_%d.h5' % (ratio)
    folder_name = 'ratio_%d' % (ratio)

    #load trained model 
    %cd /content/drive/My Drive/NN\ Input/Doppler/Train\ Data
    model = tf.keras.models.load_model(model_name)

    %cd /content/drive/My Drive/NN\ Input/Doppler/Test\ Data

    %cd $main_folder
    %cd $folder_name 
    
    for i in range(1,11): #loop over 10 test sequences 
        matname = 'DVBSAPSK_pred_ratio_%d_%d' % (ratio, i)

        mat = spio.loadmat(matname, squeeze_me=True) #load data
        x_test = mat['train_data']

        predictions = model.predict(x_test) #make predictions

        #save predictions 
        matname = 'DVBSAPSK_out_ratio_%d_%d.mat' % (ratio, i)
        spio.savemat(matname, {'pred': predictions})

    %cd ..
    %cd ..

Test robustness of NN

In [ ]:
ratios = list(range(1,16))
trained_models = list(range(1,16))

main_folder = 'robust_test'

%cd /content/drive/My Drive/NN\ Input/Doppler/Test\ Data
%mkdir $main_folder

for trained_model in trained_models:
    %cd $main_folder 
    main_folder_store = 'trained_model_%d_robust' % (trained_model)
    #folder for storing predictions from trained model at set alpha
    %mkdir $main_folder_store

    #load trained model 
    %cd /content/drive/My Drive/NN\ Input/Doppler/Train\ Data
    model_name = 'model_dopp_ratio_%d.h5' % (trained_model)
    model = tf.keras.models.load_model(model_name)



    for ratio in ratios: 
        #naviagte to test data folder
        %cd /content/drive/My Drive/NN\ Input/Doppler/Test\ Data
        #load data to predict on
        folder_name = 'ratio_%d' % (ratio)
        %cd $folder_name

        for i in range(1,11): #loop over 10 test sequences 
            matname = 'DVBSAPSK_pred_ratio_%d_%d' % (ratio, i)

            mat = spio.loadmat(matname, squeeze_me=True) #load data
            x_test = mat['train_data']

            predictions = model.predict(x_test) #make predictions

            #save predictions 
            %cd /content/drive/My Drive/NN\ Input/Doppler/Test\ Data
            %cd $main_folder 
            %cd $main_folder_store

            matname = 'DVBSAPSK_out_ratio_%d_%d.mat' % (ratio, i)
            spio.savemat(matname, {'pred': predictions})

            %cd /content/drive/My Drive/NN\ Input/Doppler/Test\ Data
            %cd $folder_name
        %cd ..